In [26]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import chi2

def likelihood_ratio_test(Y, X_full, C, m):
    """
    Teste da razão de verossimilhança para hipóteses lineares gerais.

    Y :  vetor resposta (n x 1)
    X_full : matriz de preditoras do modelo completo (n x p)
    C : matriz de restrição (q x p)
    m : vetor de restrição (q x 1)

    Retorna:
    - estatística de teste
    - p-valor
    """
    # Ajuste do modelo completo
    modelo_irrestrito = sm.OLS(Y, X_full).fit()
    logveros_irrestrita = modelo_irrestrito.llf  # log-verossimilhança do modelo completo

    # Ajuste do modelo reduzido (com restrição Cβ = m)
    X = np.asarray(X_full)
    Y = np.asarray(Y).reshape(-1, 1)
    C = np.asarray(C)
    m = np.asarray(m).reshape(-1, 1)

    XtX_inv = np.linalg.inv(X.T @ X)
    beta_hat = XtX_inv @ X.T @ Y
    lambda_ = np.linalg.inv(C @ XtX_inv @ C.T) @ (C @ beta_hat - m)
    beta_restricted = beta_hat - XtX_inv @ C.T @ lambda_

    residuos = Y - X @ beta_restricted
    sigma2 = (residuos.T @ residuos) / len(Y)
    logveros_restrita = -0.5 * len(Y) * (np.log(2 * np.pi * sigma2) + 1)

    # Estatística do teste
    W = 2 * (logveros_irrestrita - logveros_restrita.item())
    df = C.shape[0]
    p_value = 1 - chi2.cdf(W, df)

    return W, p_value


In [27]:
import numpy as np

# Simulando dados
np.random.seed(42)
n = 100
X1 = np.random.normal(size=n)
X = sm.add_constant(X1)
beta = np.array([[2], [3]])  # intercepto = 2, coef = 3
Y = X @ beta + np.random.normal(size=(n,1))

# Hipótese: H0: β1 = 0
R = np.array([[0, 1]])
r = np.array([0])

# Executa o teste
estat, p = likelihood_ratio_test(Y, X, R, r)
print(f"Estatística do teste: {estat:.4f}")
print(f"p-valor: {p:.4f}")


Estatística do teste: 214.4898
p-valor: 0.0000


In [28]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/cibelerusso/IntroducaoaInferenciaEstatistica/main/Dados/kc_house_data.csv')
df.head()

df.rename(columns={
    'id': 'id',
    'date': 'data',
    'price': 'preco',
    'bedrooms': 'quartos',
    'bathrooms': 'banheiros',
    'sqft_living': 'area_util',
    'sqft_lot': 'area_total',
    'floors': 'andares',
    'waterfront': 'frente_para_agua',
    'view': 'vista',
    'condition': 'condicao',
    'grade': 'classificacao',
    'sqft_above': 'area_sobre_o_nivel_do_chao',
    'sqft_basement': 'area_porao',
    'yr_built': 'ano_construcao',
    'yr_renovated': 'ano_reforma',
    'zipcode': 'cep',
    'lat': 'latitude',
    'long': 'longitude',
    'sqft_living15': 'area_util_vizinhanca',
    'sqft_lot15': 'area_total_vizinhanca'
}, inplace=True)

df['preco_transformado'] = (pow(df['preco'],-0.234) - 1)/(-0.234)


In [29]:
# Ajusta o modelo de regressão linear múltipla para o preço das casas com duas preditoras
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

modelo = ols('preco_transformado ~ area_util', data=df)
res = modelo.fit()


In [30]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:     preco_transformado   R-squared:                       0.456
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                 1.809e+04
Date:                Thu, 10 Apr 2025   Prob (F-statistic):               0.00
Time:                        19:56:02   Log-Likelihood:                 56032.
No. Observations:               21613   AIC:                        -1.121e+05
Df Residuals:                   21611   BIC:                        -1.120e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.0327      0.000   1.32e+04      0.0

In [31]:
import numpy as np

# Hipótese: H0: β1 = 0
C = np.array([[0, 1]])
m = np.array([0])

Y = df['preco_transformado']

X = sm.add_constant(df[['area_util']])

# Executa o teste
estat, p = likelihood_ratio_test(Y, X, C, m)
print(f"Estatística do teste: {estat:.4f}")
print(f"p-valor: {p:.4f}")

Estatística do teste: 13143.1903
p-valor: 0.0000


In [32]:
# Ajusta o modelo de regressão linear múltipla para o preço das casas com duas preditoras
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

modelo = ols('preco_transformado ~ area_util + classificacao', data=df)
res = modelo.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:     preco_transformado   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                 1.214e+04
Date:                Thu, 10 Apr 2025   Prob (F-statistic):               0.00
Time:                        19:56:02   Log-Likelihood:                 57598.
No. Observations:               21613   AIC:                        -1.152e+05
Df Residuals:                   21610   BIC:                        -1.152e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         3.9835      0.001   4455.258

In [33]:
import numpy as np

# Hipótese: H0: β1 = 0
C = np.array([[0, 1, 0]])
m = np.array([0])

Y = df['preco_transformado']

X = sm.add_constant(df[['area_util', 'classificacao']])

# Executa o teste
estat, p = likelihood_ratio_test(Y, X, C, m)
print(f"Estatística do teste: {estat:.4f}")
print(f"p-valor: {p:.4f}")

Estatística do teste: 2297.6770
p-valor: 0.0000
